In [1]:
using BenchmarkTools

In [2]:
W = trues(30000, 30000)
W[rand(1:length(W), round(Int, √length(W)))] .= 0
W

30000×30000 BitMatrix:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1   

In [3]:
function get_S(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = Set(zeroidxs[:,1])
    S2 = Set(zeroidxs[:,2])
    return S1, S2
end

get_S (generic function with 1 method)

In [4]:
@btime get_S($W)

  1.417 s (37 allocations: 3.04 MiB)


(Set([29965, 11950, 1703, 12427, 7685, 3406, 28804, 28900, 27640, 23970  …  21616, 12084, 1711, 15380, 17984, 10808, 25754, 19700, 8546, 4159]), Set([29965, 12427, 7685, 18374, 3406, 28804, 23970, 27640, 28576, 1090  …  21616, 1711, 15380, 15271, 4724, 19700, 8546, 4159, 8992, 24824]))

In [5]:
function get_S1(W)
    zeroidxs = getindex.(findall(iszero, W), [1 2])
    S1 = unique(zeroidxs[:,1])
    S2 = unique(zeroidxs[:,2])
    return S1, S2
end

get_S1 (generic function with 1 method)

In [6]:
@btime get_S1($W)

  1.405 s (107 allocations: 4.42 MiB)


([28964, 9239, 3629, 28630, 18881, 20153, 560, 27587, 14451, 14695  …  8650, 25408, 17539, 10018, 21219, 8738, 5215, 8790, 2839, 25963], [1, 3, 4, 6, 10, 13, 14, 15, 16, 18  …  29987, 29988, 29990, 29992, 29993, 29995, 29996, 29997, 29998, 29999])

In [7]:
sort.(unique.(collect.(get_S(W)))) == sort.(get_S1(W))

true

In [8]:
function get_S2(W)
    m, n = size(W)
    b1, b2 = fill(false, m), fill(false, n)
    Threads.@threads for j in 1:n
        for i in 1:n
            @inbounds if W[i, j] == 0
                b1[i] = b2[j] = true
            end
        end
    end
    b1, b2
end

get_S2 (generic function with 1 method)

In [9]:
@btime get_S2($W)

  74.563 ms (65 allocations: 64.98 KiB)


(Bool[1, 1, 1, 0, 1, 1, 1, 0, 0, 0  …  0, 0, 1, 1, 0, 0, 1, 1, 0, 1], Bool[1, 0, 1, 1, 0, 1, 0, 0, 0, 1  …  0, 1, 1, 0, 1, 1, 1, 1, 1, 0])

In [10]:
get_S3(W) = findall.(get_S2(W))

get_S3 (generic function with 1 method)

In [11]:
@btime get_S3($W)

  74.948 ms (69 allocations: 361.20 KiB)


([1, 2, 3, 5, 6, 7, 12, 14, 15, 16  …  29983, 29985, 29987, 29988, 29990, 29993, 29994, 29997, 29998, 30000], [1, 3, 4, 6, 10, 13, 14, 15, 16, 18  …  29987, 29988, 29990, 29992, 29993, 29995, 29996, 29997, 29998, 29999])

In [12]:
sort.(get_S1(W)) == get_S3(W)

true

In [13]:
get_S(W) |> typeof

Tuple{Set{Int64}, Set{Int64}}

In [14]:
get_S1(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}

In [15]:
get_S2(W) |> typeof

Tuple{Vector{Bool}, Vector{Bool}}

In [16]:
get_S3(W) |> typeof

Tuple{Vector{Int64}, Vector{Int64}}